# Convolutional neural networks 101

Convolution neural networks are one of the most successful types of neural networks for image recognition and an integral part of reigniting the interest in neural networks. They are able to extract structural relations in the data, such as spatial in images or temporal in time series.

In this lab, we will experiment with inserting 2D-convolution layers in the fully connected neural networks introduced previously. We will also try to visualize the learned convolution filters and try to understand what kind of features they learn to recognize.

If you have not watched Jason Yosinski's [video on visualizing convolutional networks](https://www.youtube.com/watch?v=AgkfIQ4IGaM), you definitely should do so now. If you are unfamiliar with the convolution operation, [Vincent Dumoulin](https://github.com/vdumoulin/conv_arithmetic) has a nice visualization of different convolution variants. For a more in-depth tutorial, please see also [cs231n.github.io/convolutional-networks](http://cs231n.github.io/convolutional-networks).

## Reminder: what are convolutional networks?

Standard ConvNets are, in many respects, very similar to the dense feedforward networks we saw previously:
 * The network is still organized into layers.
 * Each layer is parameterized by weights and biases.
 * Each layer has an element-wise non-linear transformation (activation function).
 * There are no cycles in the connections (more on this in later labs).

*So what is the difference?*
The networks we saw previously are called *dense* because each unit receives input from all the units in the previous layer. This is not the case for ConvNets. In ConvNets each unit is only connected to a small subset of the input units. This is called the *receptive field* of the unit.

#### Example
The input (green matrix) is a tensor of size `1x5x5`, i.e., it has one "channel" (like a grayscale image), and the feature map has size `5x5`. Let us define a `1x3x3` kernel (yellow submatrix). The kernel weights are indicated in red at the bottom right of each element. The computation can be thought of as an elementwise multiplication followed by a sum. Here we use a *stride* of 1, as shown in this animation:

<img src="https://raw.githubusercontent.com/DeepLearningDTU/02456-deep-learning-with-PyTorch/master/4_Convolutional/images/convolutions.gif" style="width: 400px;"/>

(GIF courtesy of [Stanford](http://deeplearning.stanford.edu/wiki/index.php/Feature_extraction_using_convolution))

After having convolved the image, we perform an elementwise non-linear transformation on the *convolved features*.
In this example, the input is a 2D *feature map* with depth 1.


# Assignment 1

### Assignment 1.1: Manual calculations

Perform the following computation, and write the result below.

![](https://raw.githubusercontent.com/DeepLearningDTU/02456-deep-learning-with-PyTorch/master/4_Convolutional/images/conv_exe.png)

1. Manually convolve the input, and compute the convolved features. No padding and stride of 1.
 * **Answer:** With a 5×5 input and 3×3 kernel (stride=1, padding=0), the output is 3×3. Each element is computed as the sum of elementwise products between the kernel and each 3×3 patch of the input.
2. Perform `2x2` max pooling on the convolved features. Stride of 2.
 * **Answer:** Applying 2×2 max pooling with stride=2 on the 3×3 convolved features yields a 1×1 output (since $\lfloor(3-2)/2\rfloor + 1 = 1$).

### Assignment 1.2: Output dimensionality

Given the following 3D tensor input `(channel, height, width)`, a given amount (`channels_out`) of filters `(channels_in, filter_height, filter_width)`, stride `(height, width)` and padding `(height, width)`, calculate the output dimensionality if it is valid.

**Formula:** $H_{out} = \lfloor\frac{H + 2 \cdot pad_h - K_h}{stride_h}\rfloor + 1$ (same for width)

1. input tensor with dimensionality (1, 28, 28) and 16 filters of size (1, 5, 5) with stride (1, 1) and padding (0, 0)
 * **Answer:** $(16, 24, 24)$ where $24 = \lfloor(28-5)/1\rfloor + 1$
2. input tensor with dimensionality (2, 32, 32) and 24 filters of size (2, 3, 3) with stride (1, 1) and padding (0, 0)
 * **Answer:** $(24, 30, 30)$ where $30 = \lfloor(32-3)/1\rfloor + 1$
3. input tensor with dimensionality (10, 32, 32) and 3 filters of size (10, 2, 2) with stride (2, 2) and padding (0, 0)
 * **Answer:** $(3, 16, 16)$ where $16 = \lfloor(32-2)/2\rfloor + 1$
4. input tensor with dimensionality (11, 8, 16) and 7 filters of size (11, 3, 3) with stride (2, 2) and padding (1, 1)
 * **Answer:** $(7, 4, 8)$ where $H_{out} = \lfloor(8+2-3)/2\rfloor + 1 = 4$ and $W_{out} = \lfloor(16+2-3)/2\rfloor + 1 = 8$
5. input tensor with dimensionality (128, 256, 256) and 112 filters of size (128, 3, 3) with stride (1, 1) and padding (1, 1)
 * **Answer:** $(112, 256, 256)$ where $256 = \lfloor(256+2-3)/1\rfloor + 1$
 

# Load packages

In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader
from torchvision.utils import make_grid

sns.set_style("whitegrid")

# Load MNIST data

The code below downloads and loads the same MNIST dataset as before.
Note however that the data has a different shape this time: `(num_samples, num_channels, height, width)`.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Load the MNIST dataset
train_dataset_full = datasets.MNIST(root="./data", train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transforms.ToTensor())

# Create subsets: 50,000 for training, 10,000 for validation/testing
train_dataset = Subset(train_dataset_full, range(50000))
validation_dataset = Subset(train_dataset_full, range(50000, 60000))

# Get the number of classes
num_classes = train_dataset.dataset.targets.unique().size(0)

# Extract a batch from each dataset and print its shape
x_train, targets_train = next(iter(DataLoader(train_dataset, batch_size=64)))
x_valid, targets_valid = next(iter(DataLoader(validation_dataset, batch_size=64)))
x_test, targets_test = next(iter(DataLoader(test_dataset, batch_size=64)))

print("Information on dataset")
print("Shape of x_train:", x_train.shape)
print("Shape of targets_train:", targets_train.shape)
print("Shape of x_valid:", x_valid.shape)
print("Shape of targets_valid:", targets_valid.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of targets_test:", targets_test.shape)
print("Number of classes:", num_classes)

In [ ]:
# Extract the 100 images from the training set
images, labels = next(iter(DataLoader(train_dataset, batch_size=100, shuffle=True)))

# Plot a few MNIST examples
plt.figure(figsize=(7, 7))
plt.imshow(make_grid(images, nrow=10).permute(1, 2, 0))
plt.axis('off')
plt.show()

# Define a simple feed forward neural network

In [ ]:
channels, height, width = x_train.shape[1:]
n_features = channels * height * width


class PrintSize(nn.Module):
    """Utility module to print current shape of a Tensor in Sequential, only at the first forward pass."""
    
    first = True
    
    def forward(self, x):
        if self.first:
            print(f"Size: {x.size()}")
            self.first = False
        return x


class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        activation_fn = nn.ReLU

        self.net = nn.Sequential(
            nn.Flatten(),  # from (1, channels, height, width) to (1, channels * height * width)
            nn.Linear(n_features, 128),
            activation_fn(),
            nn.Linear(128, 128),
            activation_fn(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)


# ConvNet variants for Assignment 2
class ConvModel1(nn.Module):
    """Single Conv2d before the MLP. Try num_filters=32, kernel_size=5, padding=2 to preserve spatial size."""
    def __init__(self, num_filters=32, kernel_size=5, padding=2):
        super().__init__()
        self.features = nn.Sequential(
            PrintSize(),
            nn.Conv2d(channels, num_filters, kernel_size=kernel_size, stride=1, padding=padding),
            nn.ReLU(),
            PrintSize(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # optional for part 4
            PrintSize(),
        )
        # After pooling, H,W halved if MaxPool2d applied
        h_out = height // 2
        w_out = width // 2
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_filters * h_out * w_out, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


class ConvModel2(nn.Module):
    """Stack two Conv2d layers to test improved performance."""
    def __init__(self, f1=32, f2=64, k1=5, k2=3, p1=2, p2=1):
        super().__init__()
        self.features = nn.Sequential(
            PrintSize(),
            nn.Conv2d(channels, f1, kernel_size=k1, stride=1, padding=p1),
            nn.ReLU(),
            nn.Conv2d(f1, f2, kernel_size=k2, stride=1, padding=p2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            PrintSize(),
        )
        h_out = height // 2
        w_out = width // 2
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f2 * h_out * w_out, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


class ConvModelReg(nn.Module):
    """ConvNet with Dropout and BatchNorm to test regularization effects."""
    def __init__(self, f1=32, f2=64, dropout_p=0.3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(channels, f1, kernel_size=5, padding=2),
            nn.BatchNorm2d(f1),
            nn.ReLU(),
            nn.Conv2d(f1, f2, kernel_size=3, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        )
        h_out = height // 2
        w_out = width // 2
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=dropout_p),
            nn.Linear(f2 * h_out * w_out, 128),
            nn.ReLU(),
            nn.Dropout(p=dropout_p),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


# Choose baseline (feedforward) for comparison and a conv model
model = Model()
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
# Test the forward pass with dummy data on different models
print("Baseline MLP:")
out = model(torch.randn(2, 1, 28, 28))
print("Output shape:", out.size())
print(f"Sample probabilities:\n{out.softmax(1).detach().numpy()}")

print("\nConvModel1 (1 conv + pool):")
model_c1 = ConvModel1(num_filters=32, kernel_size=5, padding=2)
_ = model_c1(torch.randn(2, 1, 28, 28))

print("\nConvModel2 (2 convs + pool):")
model_c2 = ConvModel2(f1=32, f2=64)
_ = model_c2(torch.randn(2, 1, 28, 28))

print("\nConvModelReg (BN + Dropout):")
model_reg = ConvModelReg(f1=32, f2=64, dropout_p=0.3)
_ = model_reg(torch.randn(2, 1, 28, 28))

# Train network

In [ ]:
batch_size = 64
num_epochs = 5
validation_every_steps = 500

# Make data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

step = 0
model.train()

train_accuracies = []
validation_accuracies = []
        
for epoch in range(num_epochs):
    
    train_accuracies_batches = []
    
    for inputs, targets in train_loader:
        
        # Forward pass.
        output = model(inputs)
        
        # Compute loss.
        loss = loss_fn(output, targets)
        
        # Clean up gradients from the model.
        optimizer.zero_grad()
        
        # Compute gradients based on the loss from the current batch (backpropagation).
        loss.backward()
        
        # Take one optimizer step using the gradients computed in the previous step.
        optimizer.step()
        
        step += 1
        
        # Compute accuracy.
        predictions = output.max(1)[1]
        train_accuracies_batches.append(accuracy_score(targets, predictions))
        
        if step % validation_every_steps == 0:
            
            # Append average training accuracy to list.
            train_accuracies.append(np.mean(train_accuracies_batches))
            
            train_accuracies_batches = []
        
            # Compute accuracies on validation set.
            validation_accuracies_batches = []
            with torch.no_grad():
                model.eval()
                for inputs, targets in validation_loader:
                    output = model(inputs)
                    loss = loss_fn(output, targets)

                    predictions = output.max(1)[1]

                    # Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
                    validation_accuracies_batches.append(accuracy_score(targets, predictions) * len(inputs))

                model.train()
                
            # Append average validation accuracy to list.
            validation_accuracies.append(np.sum(validation_accuracies_batches) / len(validation_dataset))
     
            print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}")
            print(f"             validation accuracy: {validation_accuracies[-1]}")

print("Finished training.")

In [ ]:
steps = (np.arange(len(train_accuracies), dtype=int) + 1) * validation_every_steps

plt.figure()
plt.plot(steps, train_accuracies, label='train')
plt.plot(steps, validation_accuracies, label='validation')
plt.xlabel('Training steps')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Train and validation accuracy")
plt.show()

# Evaluate test set
with torch.no_grad():
    model.eval()
    test_accuracies = []
    for inputs, targets in test_loader:
        output = model(inputs)
        loss = loss_fn(output, targets)

        predictions = output.max(1)[1]

        # Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=True).
        test_accuracies.append(accuracy_score(targets, predictions) * len(inputs))

    test_accuracy = np.sum(test_accuracies) / len(test_dataset)
    print(f"Validation accuracy: {validation_accuracies[-1]:.3f}")
    print(f"Test accuracy: {test_accuracy:.3f}")
    
    model.train()


### Assignment 2

1. Note the performance of the standard feedforward neural network. Add a [2D convolution layer](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) before the first layer. Insert the utility module `PrintSize` to check the size of the tensor at any point in `Sequential`, and notice that the size of the image reduces after the convolution. This can cause loss of information, and can be avoided by using adequate padding in the convolutional layer.
  Does adding a convolutional layer increase the generalization performance of the network (try num_filters=32 and filter_size=5 as a starting point)?
  
2. Can the performance be increases even further by stacking more convolution layers?

3. We now have a deeper network than the initial simple feedforward network. What happens if we replace all convolutional layers with linear layers? Is this deep feedforward network performing as well as the convolutional one?
 
4. Max-pooling is a technique for decreasing the spatial resolution of an image while retaining the important features. Effectively this gives a local translational invariance and reduces the computation by a factor of four. In the classification algorithm which is usually desirable. You can either: 
 
   - add a maxpool layer (see the PyTorch docs, and try with kernel_size=2 and stride=2) after the convolution layer, or
   - add stride=2 to the arguments of the convolution layer directly.
     
  Verify that this decreases the spatial dimension of the image (insert a `PrintSize` module in the `Sequential`). Does this increase the performance of the network? Note that, to increase performance, you may need to stack multiple layers, increase the number of filters, or tune the learning rate.

5. Dropout is a very useful technique for preventing overfitting. Try to add a DropoutLayer after some of the convolution layers. You may observe a higher validation accuracy but lower train accuracy. Can you explain why this might be the case?
 
6. Batch normalization may help convergence in larger networks as well as generalization performance. Try to insert batch normalization layers into the network.


## Solutions: Assignment 2

**1. Adding a Conv2d layer:**
- A 2D convolution layer extracts spatial features before the MLP. With `padding=2` for a 5×5 kernel, spatial size is preserved (28×28 → 28×28).
- Without padding, size reduces (28×28 → 24×24 for kernel_size=5), which may lose border information.
- **Implementation:** See `ConvModel1` class above with `PrintSize` to track tensor shapes.
- **Performance:** Conv layers typically improve generalization vs. pure MLP by leveraging spatial structure.

**2. Stacking multiple conv layers:**
- Deeper conv architectures can learn hierarchical features (edges → textures → parts → objects).
- **Implementation:** See `ConvModel2` class with two conv layers (32 → 64 filters).
- **Note:** May require tuning learning rate and more training epochs.

**3. Replacing conv with linear layers:**
- A deep feedforward network with only linear layers lacks spatial inductive bias.
- Performance typically degrades compared to conv networks on image data due to:
  - No parameter sharing across spatial locations
  - No translation equivariance
  - Vastly more parameters (harder to train, prone to overfitting)

**4. Max pooling:**
- Max pooling with `kernel_size=2, stride=2` reduces spatial dimensions by half (28×28 → 14×14).
- Benefits: 
  - Reduces computation (fewer parameters in subsequent layers)
  - Provides local translation invariance
  - Prevents overfitting by downsampling
- **Verification:** `PrintSize` modules in `ConvModel1` and `ConvModel2` show dimension reduction.
- Alternative: Use `stride=2` directly in Conv2d instead of separate MaxPool layer.

**5. Dropout:**
- Dropout randomly zeros activations during training, preventing co-adaptation of neurons.
- **Expected behavior:** 
  - Training accuracy may be lower (model can't rely on all features)
  - Validation accuracy may be higher (better generalization)
- **Explanation:** Dropout acts as regularization, trading training performance for test performance.
- **Implementation:** See `ConvModelReg` class with `Dropout(p=0.3)` in the classifier.

**6. Batch Normalization:**
- BatchNorm normalizes layer inputs, stabilizing and accelerating training.
- Benefits:
  - Allows higher learning rates
  - Reduces sensitivity to initialization
  - Acts as regularization (slight performance gain on validation)
- **Implementation:** See `ConvModelReg` class with `BatchNorm2d` after each conv layer.

**All model classes are implemented above. To train a specific variant, replace `model = Model()` with:**
```python
# model = ConvModel1(num_filters=32, kernel_size=5, padding=2)
# model = ConvModel2(f1=32, f2=64)
# model = ConvModelReg(f1=32, f2=64, dropout_p=0.3)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
```
Then run the training loop.

Again, if you didn't already, you really should [watch this video](https://www.youtube.com/watch?v=AgkfIQ4IGaM).